In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from explorer_helper import *

In [22]:
df['captured_out'] = df.apply(lambda r: r.exp.captured_out, axis=1)
df['heartbeat'] = df.apply(lambda r: r.exp.heartbeat, axis=1)
df['start'] = df.apply(lambda r: r.exp.start_time, axis=1)
df['duration'] = df.apply(lambda r: (r.exp.heartbeat - r.exp.start_time) if not pd.isnull(r.exp.heartbeat) else 0, axis=1)
df['double'] = df.apply(lambda r: r.config.model.kwargs.model.kwargs.get('use_double_precision', False), axis=1)
df['PCG'] = df.apply(lambda r: r.config.model.kwargs.model.kwargs.get('precond_size', 10), axis=1)
df['newest loss'] = df.apply(lambda r: r.exp.metrics.get('DKLGPModel.training.loss', pd.Series([None])).iloc[-1], axis=1)
df['M'] = df.apply(get_M, axis=1)

df[['status', 'M', 'double', 'N', 'PCG', 'id', 'model', 'duration', 'newest loss' ,'tag']]

NameError: name 'df' is not defined

# Server Executation

In [21]:
settings.MODE = settings.MODES.SERVER
settings.SAVE = True

SGPR = {
    'name': 'SGPR',
    'kwargs': {
        'learning_rate': 0.1,
        'n_iter': 500,
        'nn_kwargs': {'layers': None},
        'gp_kwargs': {'inducing_points': 100},
        'use_double_precision': True,
    },
}

DKLSGPR = {
    'name': 'SGPR',
    'kwargs': {
        'do_pretrain': True,
        'pretrain_n_iter': 500,
        'learning_rate': 0.01,
        'n_iter': 300,
        'nn_kwargs': {'layers': [1]},
        'gp_kwargs': {'inducing_points': 100},
        'use_double_precision': True,
    },
}

transformer = {
    'name': 'ActiveSubspace',
    'kwargs': {
        'output_dim': 1
    }
}

models = [
    DKLSGPR,
    {
        'name': 'TransformerModel',
        'kwargs': {
            'transformer': transformer,
            'prob_model': SGPR,
        },
    },
]

functions = [
    {'name': 'ActiveSubspaceArbitrary1D', 'kwargs': {'D': 100}},
    {'name': 'ActiveSubspaceArbitrary1D', 'kwargs': {'D': 1000}},
 ]

for func in functions:
    for model in models:
        # Helper function to wraps all model in a NormalizerModel
        config = normalize_config({
            'tag': 'demo',
            'obj_func': func,
            'model': model,
            'gp_use_derivatives': model.get('name') == 'TransformerModel',
            'gp_samples': 20 * func.get('kwargs').get('D'),
        })
        run = execute(config_updates=config)

['ssh', 'simba', 'cd mthesis; sbatch hpc.sh \'python\' \'runner.py\' \'with\' \'tag="demo"\' \'obj_func={"name": "ActiveSubspaceArbitrary1D", "kwargs": {"D": 100}}\' \'model={"name": "NormalizerModel", "kwargs": {"model": {"name": "SGPR", "kwargs": {"do_pretrain": True, "pretrain_n_iter": 500, "learning_rate": 0.01, "n_iter": 300, "nn_kwargs": {"layers": [1]}, "gp_kwargs": {"inducing_points": 100}, "use_double_precision": True}}, "normalize_input": True, "normalize_output": True}}\' \'gp_use_derivatives=False\' \'gp_samples=2000\'']
b'Submitted batch job 73235\n'
['ssh', 'simba', 'cd mthesis; sbatch hpc.sh \'python\' \'runner.py\' \'with\' \'tag="demo"\' \'obj_func={"name": "ActiveSubspaceArbitrary1D", "kwargs": {"D": 100}}\' \'model={"name": "NormalizerModel", "kwargs": {"model": {"name": "TransformerModel", "kwargs": {"transformer": {"name": "ActiveSubspace", "kwargs": {"output_dim": 1}}, "prob_model": {"name": "SGPR", "kwargs": {"learning_rate": 0.1, "n_iter": 500, "nn_kwargs": {"la

# Data exploration

In [20]:
exps_rows_df = get_df(**{'config.tag': "demo"})
exps_rows_df

KeyError: 'exp_hash'

In [6]:
temp_df = aggregate_results(exps_rows_df)
temp_df = exps_rows_df

def get_lr(row):
    kwargs = row.config.model.kwargs.model.kwargs.prob_model.get('kwargs', None)
    if kwargs is None:
        kwargs = row.config.model.kwargs.model.kwargs
    return kwargs.get('learning_rate', None)

def get_n_iter(row):
    kwargs = row.config.model.kwargs.model.kwargs.prob_model.get('kwargs', None)
    if kwargs is None:
        kwargs = row.config.model.kwargs.model.kwargs
    return kwargs.get('n_iter', -1)

def get_layers(row):
    layers = row.config.model.kwargs.model.kwargs.nn_kwargs.get('layers', None)
    if layers is None:
        layers = row.config.model.kwargs.model.kwargs.prob_model.kwargs.nn_kwargs.get('layers', None)
    return layers

temp_df['f.D'] = temp_df.apply(lambda r: r.config.obj_func.kwargs.D, axis=1)
temp_df['n_iter'] = temp_df.apply(get_n_iter, axis=1)
temp_df['lr'] = temp_df.apply(get_lr, axis=1)
temp_df['pretrain'] = temp_df.apply(lambda r: r.config.model.kwargs.model.kwargs.get('do_pretrain', False), axis=1)
temp_df['layers'] = temp_df.apply(get_layers, axis=1)
temp_df['layers'] = temp_df.apply(lambda row: tuple(row.layers) if row.layers is not None else (), axis=1)

#temp_df = temp_df[temp_df['n_iter'] == 8000]
pd.options.display.max_rows = 999
view_df(temp_df, indexes=['f.D', 'f', 'model', 'layers', 'lr', 'n_iter', 'pretrain', 'N'], cols=['result.rmse', 'result.max_err', 'result.time:training']).sort_index()

KeyError: "None of [Index(['result.rmse', 'result.max_err', 'result.time:training'], dtype='object')] are in the [columns]"

In [19]:
temp_df[temp_df['id']==3535].iloc[0]

N                                                          2000
acq                                                        None
bo                                                        False
config        {'exp_hash': '2ac060b396c53eaf540260b2ff0a990e...
exp                             Experiment(id=3535, name=lions)
f                                  ActiveSubspaceArbitrary1D100
id                                                         3535
model                                                   N<SGPR>
model2                                                     None
model_hash             6ae21eeb7af0ebcd70651b9eb9b9feef34284b09
name                       N<SGPR> ActiveSubspaceArbitrary1D100
status                                                  RUNNING
tag                                                        demo
f.D                                                         100
n_iter                                                      300
lr                                      

# Local Execution